In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from pandas import DataFrame
import re

In [5]:
url = 'https://www.imdb.com/feature/genre/?ref_=nv_ch_gr'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
ge = ["https://www.imdb.com" + i.get('href') for i in soup.select('div[class="table-cell primary"] > a')]

In [7]:
list=[]
for genre_link in ge[-24:]:
    soup = BeautifulSoup(requests.get(genre_link).text, "html.parser")
    while True:
        movies = soup.find_all('div', class_='lister-item-content')
        titles = [i.h3.a.text for i in movies]
        years = [i.select('span[class="lister-item-year text-muted unbold"]')[0].text for i in movies]
        certificate = [i.select('span.certificate') for i in movies]
        certificate = ["" if i==[] else i[0].text for i in certificate ]
        genre = [''.join(i.select('span.genre')[0].text.split()) for i in movies]
        runtime = [i.select('span.runtime') for i in movies]
        runtime = ["" if i==[] else i[0].text for i in runtime ]
        rating = [i.select('div[class="inline-block ratings-imdb-rating"] > strong') for i in movies]
        rating = ["" if i==[] else i[0].text for i in rating ]
        metascore = [i.select('div[class="inline-block ratings-metascore"] > span') for i in movies]
        metascore = ["" if i==[] else ''.join(i[0].text.split()) for i in metascore ]
        description = [i.select('p[class="text-muted"]')[1].text[1:] for i in movies]
        votes = [i.select('p[class="sort-num_votes-visible"] > span[name="nv"]') for i in movies]
        votes = ["" if i==[] else i[0].get('data-value') for i in votes ]
        gross = [i.select('p[class="sort-num_votes-visible"] > span[name="nv"]') for i in movies]
        gross = ["" if len(i)<2 else i[1].get('data-value') for i in gross ]
        texts = ["" if i==[] else ''.join(i.select('p[class=""]')[0].text.split()) for i in movies]
        Director=[]
        Stars=[]
        for i in texts:
            if 'Stars:' in i:
                index=i.index('Stars:')
                Stars.append(i[index+6:])
                Director.append("") if index==0 else Director.append(i[9:index-1])
            else:
                if 'Director:' in i:
                    Stars.append('')
                    Director.append(i[9:])
                else:
                    Stars.append('')
                    Director.append('')
        try:
            for index in range(0, len(movies)):
                data = {"movie_title": titles[index],
                        "year": years[index],
                        "certificate": certificate[index],
                        "runtime": runtime[index],
                        "genre": genre[index],
                        "ratings": rating[index],
                        "metascore": metascore[index],
                        "description": description[index],
                        "Director": Director[index],
                        "stars": Stars[index],
                        "votes": votes[index],
                        "gross": gross[index]
                        }
                list.append(data)
        except:
            print(titles[0])
        print(len(list))
        try:
            next_url = soup.select_one('a[class="lister-page-next next-page"]').get('href')
            soup = BeautifulSoup(requests.get("https://www.imdb.com"+next_url).text, "html.parser")
        except:
            break

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2138
2188
2238
2288
2338
2388
2438
2488
2538
2588
2638
2688
2738
2788
2838
2888
2938
2988
3038
3088
3138
3188
3238
3288
3338
3388
3438
3488
3538
3588
3638
3688
3726
3776
3826
3876
3926
3976
4026
4076
4123
4173
4223
4273
4323
4373
4423
4473
4523
4573
4623
4673
4723
4739
4789
4839
4889
4939
4989
5039
5089
5139
5189
5239
5289
5339
5389
5439
5489
5539
5589
5639
5689
5739
5789
5839
5889
5939
5989
6039
6089
6139
6189
6239
6289
6339
6389
6439
6489
6539
6589
6639
6689
6739
6789
6839
6889
6939
6989
7039
7089
7139
7189
7239
7289
7339
7389
7439
7489
7539
7589
7639
7689
7739
7789
7839
7889
7939
7989
8039
8057
8107
8157
8207
8257
8307
8357
8407
8457
8507
8557
8607
8657
8707
8757
8807
8857
8907
8957
9007
9057
9107
9157
9207
9257
9307
9357
9407
9457
9507
9557
9607
9657
9707
9757
9807
9857
9872
9922
9972
10022
1007

In [85]:
df = pd.DataFrame(list)
df['ratings'] = df['ratings'].apply(lambda r: None if r == '' else float(r))
df['runtime'] = df['runtime'].apply(lambda t: None if t == '' else int(t[:-4].replace(",", "")))
df['gross'] = df['gross'].apply(lambda g: -1 if g == '' else int(g.replace(",", "")))
df['metascore'] = df['metascore'].apply(lambda m: None if m == '' else int(m))
def start_y(runtime):
    r1 = [i.split('–') for i in re.findall('\((.*?)\)', str(runtime))]
    for i in r1:
        if len(i)>=1 and len(i[0])==4:
            try:
                return int(i[0])
            except:
                continue
    return 0
df['start_year'] = df[['year']].apply(start_y, axis=1)
median = df['start_year'].median()
#df['start_year'] = df[['start_year']].apply((lambda sy: median if (sy == 0) else sy), axis=1)
#df['new_gross'] = df.apply(lambda x:df[df['start_year']==x['start_year']]['gross'].mean() if (x['gross'] == -1) else x['gross'],axis=1)
#df[['gross','start_year']].apply(lambda g: [(df[df['start_year']==g[1]]["gross"].mean()),g[1]] if g[0] else g, axis=1)


In [92]:
f= (lambda sy: median if (sy == 0) else sy)
df['start_year'] = df[['start_year']].apply(f, axis=1)
#df['new_gross'] = df.apply(lambda x:df[df['start_year']==x['start_year']]['gross'].mean() if (x['gross'] == -1) else x['gross'],axis=1)
#df[['gross','start_year']].apply(lambda g: [(df[df['start_year']==g[1]]["gross"].mean()),g[1]] if g[0] else g, axis=1)


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [93]:
df

,movie_title,year,certificate,runtime,genre,ratings,metascore,description,Director,stars,votes,gross,start_year
0,The Dark Knight,(2008),PG-13,152.0,"Action,Crime,Drama",9.0,84.0,When the menace known as the Joker wreaks havo...,ChristopherNolan,"ChristianBale,HeathLedger,AaronEckhart,Michael...",2678695,534858444,2008
1,The Lord of the Rings: The Return of the King,(2003),PG-13,201.0,"Action,Adventure,Drama",9.0,94.0,Gandalf and Aragorn lead the World of Men agai...,PeterJackson,"ElijahWood,ViggoMortensen,IanMcKellen,OrlandoB...",1862320,377845905,2003
2,The Lord of the Rings: The Fellowship of the Ring,(2001),PG-13,178.0,"Action,Adventure,Drama",8.8,92.0,A meek Hobbit from the Shire and eight compani...,PeterJackson,"ElijahWood,IanMcKellen,OrlandoBloom,SeanBean",1891766,315544750,2001
3,Inception,(2010),PG-13,148.0,"Action,Adventure,Sci-Fi",8.8,74.0,A thief who steals corporate secrets through t...,ChristopherNolan,"LeonardoDiCaprio,JosephGordon-Levitt,ElliotPag...",2376670,292576195,2010
4,The Lord of the Rings: The Two Towers,(2002),PG-13,179.0,"Action,Adventure,Drama",8.8,87.0,While Frodo and Sam edge closer to Mordor with...,PeterJackson,"ElijahWood,IanMcKellen,ViggoMortensen,OrlandoB...",1681610,342551365,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27895,Bad Girls,(1994),R,99.0,"Action,Adventure,Romance",5.2,NaN,Four prostitutes join together to travel the O...,JonathanKaplan,"MadeleineStowe,MaryStuartMasterson,AndieMacDow...",13023,15240435,1994
27896,Blueberry,(2004),R,124.0,"Adventure,Western",5.2,NaN,"In 1870, after a brutal run-in with an outlaw ...",JanKounen,"VincentCassel,MichaelMadsen,JulietteLewis,Temu...",12051,-1,2004
27897,Wild Wild West,(1999),PG-13,106.0,"Action,Comedy,Sci-Fi",4.9,38.0,The two best special agents in the Wild West m...,BarrySonnenfeld,"WillSmith,KevinKline,KennethBranagh,SalmaHayek",162934,113805681,1999
27898,The Ridiculous 6,(2015),,119.0,"Action,Adventure,Comedy",4.8,18.0,An outlaw who was raised by Native Americans d...,FrankCoraci,"AdamSandler,TerryCrews,JorgeGarcia,TaylorLautner",50680,-1,2015
